# Feature Engineering

In diesem Notebook wird der Prozess der Feature Engineerings vorgestellt. Als Beispiel wird die Vorbereitung von Daten für einen Lineare Regressionsanalyse benutzt, und zwar der Daten zu Luftqualität und Wetterdaten für die Stadt **Hamburg**. Anhand der Wettervariablen soll die Belastung mit Feinstaub vorhergesagt werden.

Die **lineare Regression** ist ein statistisches Verfahren, mit dem der Zusammenhang zwischen einer abhängigen Variable (Zielvariable) und einer oder mehreren unabhängigen Variablen (Features) modelliert wird. Ziel ist es, eine **lineare Gleichung** zu finden, mit der man den Wert der Zielvariable aus den Werten der Features vorhersagen kann.

Die allgemeine Form einer einfachen linearen Regression (mit nur einem Feature) lautet:

$$
y = \beta_0 + \beta_1 x + \varepsilon
$$

- y: Zielvariable (z. B. Feinstaubkonzentration)  
- x: unabhängige Variable (z. B. Temperatur)  
- $\beta_0$: Achsenabschnitt (Intercept)  
- $\beta_1$: Steigung (Einfluss des Features auf das Ziel)   -->
- $\varepsilon$: Fehlerterm (Differenz zwischen Vorhersage und Realität)

Bei **multipler linearer Regression** werden mehrere Features gleichzeitig berücksichtigt:

$$
y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \dots + \beta_n x_n + \varepsilon
$$

Die lineare Regression geht davon aus, dass der Zusammenhang zwischen den Variablen **linear** ist – also, dass sich Änderungen in den Features proportional auf die Zielvariable auswirken.

**Typische Anwendungsfälle:**

- Vorhersage von Luftschadstoffkonzentrationen auf Basis von Wetterdaten  
- Erklärung, welche Faktoren welchen Einfluss auf die Zielgröße haben

Die folgenden Analysen werden zeigen, dass der Einfluss der Variablen nicht immer linear ist. Beispielsweise ändert sich der... (Fortsetzung folgt)

Diese Beobachtung motiviert den Einsatz von Feature Engineering mit dem Ziel, die vorhandenen Daten möglichst gut vorzubereiten, um sie für ein Machine-Learning-Modell benutzen zu können.

Verwendet werden verschiedene Module der Python Bibliothek **Scitkit-learn** für maschinelles Lernen, so wie **statsmodels**.

📌 **Datenstand:** `cleaned_air_quality_data_2025-03-27.csv`  
📁 **Importiert aus:** lokaler Datei (--> gitignore)


## 📚 Inhaltsverzeichnis 
(Diese Art von Inhaltsverzeichnis mit Link funktioniert leider in Notebooks nicht, weil die as JSON gespeichert werden und nicht als HTML...)

- [0. Datensatz laden](#0-datensatz-laden)
- [1. Mini-EDA Hamburg](#1-mini-eda-hamburg)
- [2. Baseline-Modell: Lineare Regression](#2-baseline-modell-lineare-regression)
- [3. Feature Engineering](#3-feature-engineering)
    - [3.1. Feature Reduction](#31-feature-reduction)
    - [3.2. Feature Transformation](#32-feature-transformation)
    - [3.3. Interaktionsterme](#33-interaktionsterme)
    - [3.4. Zeitvariablen](#34-zeitvariablen)
- [4. Fazit: Wirkung von Feature Engineering auf das Regressionsmodell](#4-fazit-wirkung-von-feature-engineering-auf-das-regressionsmodell)
- [5. Bonus: Weitere Luftschadstoffe als erklärende Variablen](#5-bonus-weitere-luftschadstoffe-als-erklärende-variablen)
- [6. Finales Modell: Visualisierung der Ergebnisse](#6-finales-modell-visualisierung-der-ergebnisse)
- [7. Modellvergleich](#7-modellvergleich)
    

# 0. Datensatz laden

In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from statsmodels.nonparametric.smoothers_lowess import lowess
%matplotlib inline

In [ ]:
# Settings for displaying floats
pd.set_option('display.float_format', '{:,.2f}'.format)

In [ ]:
df = pd.read_csv("data/cleaned_air_quality_data_2025-03-27.csv")
df.head()

# 1. Mini-EDA: Hamburg

In [ ]:
# Erstellen eines DataFrames für Hamburg
df_hamburg = df[df['City'].isin(['Hamburg'])]

# Überprüfung der ersten Zeilen des gefilterten DataFrames
print(df_hamburg.shape)
df_hamburg.head()

In [ ]:
df_hamburg.columns

In [ ]:
df_hamburg.describe().T

In [ ]:
df_hamburg['Co'].unique()

Die Werte für CO sind auffällig niedrig. Messfehler? Fehlende Messwerte?

Als Target für die Regressionsanalyse wird die Feinstaubbelastung (PM2.5) gewählt.

Als Features werden die Wettervariablen Durchschnittstemperatur (Tavg), Luftfeuchtigkeit (Humidity), Niederschalg (Prcp), Windgeschwindigkeit (Wspd) und Luftdruck (Pres) ausgewählt.

In [ ]:
# Auswahl der relevanten Spalten für die Analyse
df_hamburg_pm25 = df_hamburg[['Year', 'Month', 'Day', 'Pm25', 'Tavg', 'Humidity', 'Prcp', 'Wspd', 'Pres']]

# Überprüfung der ersten Zeilen des neuen DataFrames
df_hamburg_pm25.head()

In [ ]:
# Fehlende Werte in den relevanten Spalten zählen
missing_values = df_hamburg_pm25.isnull().sum()
missing_values = missing_values[missing_values > 0]
print("Missing values in relevant columns:")    
missing_values

Es ist zu vermuten, dass für Wettervariablen 'Tavg', 'Prcp', 'Wspd' und 'Pres' für 67 Tage nicht übermittelt wurden.

In [ ]:
missing_weather_rows = df_hamburg_pm25[
    df_hamburg_pm25[['Tavg', 'Prcp', 'Wspd', 'Pres']].isna().all(axis=1)
]

len(missing_weather_rows)

Über den Umgang mit NaN-Werten werden wir uns später gedanken machen, Zunächst ermitteln wir die Korrelationen zwischen den Variablen:

In [ ]:
# Berechnung der Korrelationsmatrix
correlation_matrix = df_hamburg_pm25[['Pm25', 'Tavg', 'Humidity', 'Prcp', 'Wspd', 'Pres']].corr()

# Visualisierung der Korrelationsmatrix mit 'center=0'

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', center=0)
plt.title('Korrelation zwischen PM2.5 und Wetterdaten in Hamburg')

Eine starke Korrelation zwischen dem Feinstaubwert und einer Wettervariablen ist nicht unmittelbar offensichtlich. Am stärksten ist mit -0,19 die Korrelation mit der Windgeschwindigkeit. Das bedeutet, dass in Zeiten mit wenig Wind die PM2.5-Werte tendenziell höher sind. Dies ist zu erwarten, da Wind dazu beiträgt, Luftschadstoffe zu vertreiben, während ruhiges Wetter (wenig Wind) zu einer Ansammlung von Schadstoffen führen kann.

Die Pearson-Korrelationsmatrix nimmt eine lineare Korrelation zwischen den Variablenpaaren an. Diese ist aber nicht zwingend gegeben (s. 2_eda_correlations.ipynb). Um zu sehen, wie die Korrelationen aussehen, erstellen wir Scatterplots:

In [ ]:
# Erstelle Scatterplots für jede Wettervariable im Vergleich zu PM2.5
variables = ['Tavg', 'Humidity', 'Prcp', 'Wspd', 'Pres']
plt.figure(figsize=(12, 10))

for i, var in enumerate(variables, 1):
    plt.subplot(2, 3, i)  # 2 Zeilen, 3 Spalten
    plt.scatter(df_hamburg_pm25[var], df_hamburg_pm25['Pm25'], alpha=0.6)
    plt.title(f'Scatterplot: PM2.5 vs {var}')
    plt.xlabel(var)
    plt.ylabel('PM2.5 (µg/m³)')
    plt.grid(True)

plt.tight_layout();

Nicht sicher, ob ich diesen Zweig hier weiterverfolgen möchte. Das Prinzip wird im Notebook zu Korrelationen ausführlich genug dargestellt.

# 2. Baseline-Modell: Lineare Regression

Bevor wir ein Modell für lineare Regression verwenden können, müssen wir die NaN-Werte behandeln. Da wir Datumsangaben haben, könnten wir die Werte des Vortrags, bzw. der vorhergehenden Messung imputieren. Da der Datensatz aber mehrere längere Lücken aufweist, entscheiden wir uns für eine Imputation über den Median.

In [ ]:
# Inputiere NaN-Werte mit dem Median der jeweiligen Spalte
df_hamburg_pm25.fillna(df_hamburg_pm25.median(), inplace=True)

# Überprüfung, ob NaN-Werte noch vorhanden sind
print(df_hamburg_pm25.isna().sum())

Der Dataframe enthält nun keine NaN_Werte mehr und kann für die Modellierung verwendet werden.

### Lineare Regression mit Scikit-Learn und Statsmodels

Es soll ein lineares Regressionsmodell erstellt werden, um den Zusammenhang zwischen der Feinstaubbelastung (PM2.5) und verschiedenen Wetterfaktoren zu untersuchen.

Dabei werden zwei Bibliotheken kombiniert:

- **Scikit-Learn**: zum Trainieren des Modells und zur Berechnung des Bestimmtheitsmaßes $r^2$
- **Statsmodels**: zur detaillierten Analyse des Modells (Koeffizienten, Standardfehler, p-Werte, Konfidenzintervalle, usw.)

Die verwendeten unabhängigen Variablen (Features) sind:

- Durchschnittstemperatur (`Tavg`)
- Luftfeuchtigkeit (`Humidity`)
- Niederschlag (`Prcp`)
- Windgeschwindigkeit (`Wspd`)
- Luftdruck (`Pres`)

Die abhängige Variable ist die Feinstaubkonzentration (`Pm25`).


In [ ]:
# Feature-Auswahl & Zielvariable definieren
features = ['Tavg', 'Humidity', 'Prcp', 'Wspd', 'Pres']
X = df_hamburg_pm25[features]
y = df_hamburg_pm25['Pm25']

In [ ]:
# Modelltraining
lr_model = LinearRegression()
lr_model.fit(X, y)

# Modellparameter anzeigen
print("Intercept (β₀):", lr_model.intercept_)
print("Koeffizienten (β₁...βₙ):", lr_model.coef_)
print("R² (Bestimmtheitsmaß):", lr_model.score(X, y))

In [ ]:
# Statsmodels: Regressionsanalyse mit Zusammenfassung
X_sm = sm.add_constant(X)  # Intercept hinzufügen
ols_model = sm.OLS(y, X_sm).fit()
print(ols_model.summary())

### 📊 Interpretation der Regressionsanalyse

Die lineare Regressionsanalyse wurde durchgeführt, um den Einfluss verschiedener Wetterfaktoren auf die PM2.5-Konzentration in Hamburg zu untersuchen. Die Ergebnisse zeigen Folgendes:

#### 🔹 Modellgüte

- **r^2 = 0,072**: Nur etwa **7,2 %** der Varianz der PM2.5-Werte kann durch die gewählten Wettervariablen erklärt werden. Das Modell hat also eine geringe Vorhersagekraft.
- **Adj. r^2 = 0,070**: Auch die bereinigte Variante (angepasst an die Anzahl der Prädiktoren) bestätigt die geringe Erklärungskraft.
- **F-Statistik = 43.76**, **p < 0.001**: Das Modell als Ganzes ist dennoch **statistisch signifikant**, d. h. mindestens einer der Prädiktoren trägt systematisch zur Erklärung von PM2.5 bei.

#### 🔹 Koeffizienten und Signifikanz

| Variable   | Einflussrichtung | p-Wert  | Interpretation |
|------------|------------------|---------|----------------|
| **Tavg**   | negativ          | 0.000   | Höhere Temperaturen gehen mit niedrigeren PM2.5-Werten einher. |
| **Humidity** | positiv        | 0.000   | Höhere Luftfeuchtigkeit ist mit leicht erhöhten PM2.5-Werten assoziiert. |
| **Prcp**   | negativ          | 0.000   | Niederschlag reduziert die PM2.5-Konzentration – vermutlich durch Auswaschungseffekte. |
| **Wspd**   | negativ          | 0.000   | Höhere Windgeschwindigkeit führt zu einer Verdünnung von Feinstaub. |
| **Pres**   | kein signifikanter Effekt | 0.378 | Luftdruck scheint keinen nennenswerten Einfluss auf PM2.5 zu haben. |

Alle Wettervariablen außer dem Luftdruck zeigen einen **statistisch signifikanten Einfluss** auf die Feinstaubbelastung (p < 0.05).

#### 🧭 Fazit

Auch wenn das Modell nur einen kleinen Teil der Variation erklären kann, lassen sich einige **systematische Zusammenhänge** zwischen Wetterbedingungen und PM2.5-Konzentration erkennen. Diese Ergebnisse könnten ein nützlicher Ausgangspunkt für eine weiterführende Modellierung (z. B. mit nichtlinearen oder multivariaten Ansätzen) sein.


# 3. Feature Engineering

Im Folgenden werden die im Modell verwendeten Features nacheinander bearbeitet mit dem Ziel, die Modellgüte zu verbessern. Da das Modell insgesamt sehr schwach ist, wird keine nennenswerte Verbesserung durch das Feature Engineering erwartet. Es soll aber gezeigt werden, welche Mothoden prinzipiell zur Verfügung stehen.


## 3.1. Feature Reduction

Basierend auf der vorherigen Regressionsanalyse wurde der Luftdruck (`Pres`) als **nicht signifikant** identifiziert (p = 0.378). In diesem Schritt wird `Pres` aus dem Feature-Set entfernt, um zu überprüfen, ob sich die Modellgüte dadurch verbessert oder konstant bleibt.

In [ ]:
# Neues Feature-Set ohne 'Pres'
X_reduced = df_hamburg_pm25[['Tavg', 'Humidity', 'Prcp', 'Wspd']]
y = df_hamburg_pm25['Pm25']

# Mit Statsmodels analysieren
X_reduced = sm.add_constant(X_reduced)
model_reduced = sm.OLS(y, X_reduced).fit()
print(model_reduced.summary())

#### Ergebnis:

- Das neue Modell enthält nur noch vier Wettervariablen: `Tavg`, `Humidity`, `Prcp` und `Wspd`.
- Das Bestimmtheitsmaß $R^2$ bleibt mit **0.072** unverändert.
- Auch das bereinigte $R^2$ sowie die AIC- und BIC-Werte bleiben nahezu gleich.
- Damit zeigt sich, dass der Luftdruck keinen relevanten Beitrag zur Modellgüte leistet und ohne Informationsverlust entfernt werden kann.

👉 Dies ist ein typisches Beispiel für **Feature-Selektion als Teil des Feature Engineerings**, bei dem überflüssige oder irrelevante Variablen identifiziert und entfernt werden, um Modelle robuster und interpretierbarer zu machen.

## 3.2. Feature Transformation

#### Logarithmierung der Niederschlagswerte

Da Niederschlagswerte oft viele Nullen und eine schiefe Verteilung aufweisen, wurde in diesem Schritt eine **logarithmische Transformation** durchgeführt:

$$\text{Log\_Prcp} = \log(\text{Prcp} + 1)$$


In [ ]:

# Neue Spalte mit logarithmierter Niederschlagsmenge
df_hamburg_pm25['Log_Prcp'] = np.log(df_hamburg_pm25['Prcp'] + 1)

# Neues Feature-Set (Log_Prcp statt Prcp)
X_log = df_hamburg_pm25[['Tavg', 'Humidity', 'Log_Prcp', 'Wspd']]
y = df_hamburg_pm25['Pm25']

# Modell fitten
X_log = sm.add_constant(X_log)
model_log = sm.OLS(y, X_log).fit()
print(model_log.summary())

#### Ergebnis:

- Die lineare Variable `Prcp` wurde durch `Log_Prcp` ersetzt.
- Das neue Modell zeigt ein leicht verbessertes Bestimmtheitsmaß:
  - $R^2$: von **0.072** → **0.078**
  - Adj. $R^2$: von **0.070** → **0.076**
- Auch AIC und BIC sind etwas gesunken → das neue Modell ist **effizienter**.
- Der Koeffizient von `Log_Prcp` ist mit **-3.13** stark negativ und hoch signifikant (p < 0.001), was auf eine deutliche Reduktion von PM2.5 bei zunehmendem Regen hinweist (nach log-Skalierung).

➡️ Diese Transformation ist ein klassisches Beispiel dafür, wie **Feature Engineering** helfen kann, nichtlineare Zusammenhänge **besser erfassbar zu machen** und das Modell zu verbessern.


#### Quadratischer Term für Temperaturwerte

Um zu prüfen, ob der Zusammenhang zwischen Temperatur (`Tavg`) und Feinstaubkonzentration (`Pm25`) nichtlinear ist, wurde in diesem Schritt ein **quadratischer Term** (`Tavg_squared`) ergänzt:

- Sowohl der lineare als auch der quadratische Term wurden in das Modell aufgenommen.
- Die neue Modellform erlaubt es, auch U- oder umgekehrt U-förmige Zusammenhänge abzubilden.


In [ ]:
# Quadratischer Term für Temperatur
df_hamburg_pm25['Tavg_squared'] = df_hamburg_pm25['Tavg'] ** 2

# Neues Feature-Set inkl. Tavg² und Log_Prcp
X_quad = df_hamburg_pm25[['Tavg', 'Tavg_squared', 'Humidity', 'Log_Prcp', 'Wspd']]
y = df_hamburg_pm25['Pm25']

# Modell mit quadratischer Temperaturkomponente
X_quad = sm.add_constant(X_quad)
model_quad = sm.OLS(y, X_quad).fit()
print(model_quad.summary())


#### Ergebnis:

- Das Modell zeigt eine deutliche Verbesserung:
  - $R^2$: von **0.078** → **0.122**
  - Adj. $R^2$: von **0.076** → **0.120**
  - AIC/BIC ebenfalls gesunken → effizienteres Modell
- Beide Temperatur-Terme (`Tavg` und `Tavg_squared`) sind **hoch signifikant** (p < 0.001)
- Der negative Koeffizient von `Tavg` und der positive von `Tavg_squared` deuten auf einen **U-förmigen Zusammenhang** hin:  
  Sehr niedrige und sehr hohe Temperaturen gehen mit **erhöhter Feinstaubbelastung** einher, während mittlere Temperaturen tendenziell niedrigere Werte zeigen.

➡️ Diese Erweiterung zeigt, wie sich mit gezielten Transformationen **wichtige Muster sichtbar machen** lassen, die in einem rein linearen Modell verborgen geblieben wären.

## 3.3. Interaktionsterme

In einem explorativen Schritt wird geprüft, ob eine **Interaktion zwischen Temperatur und Luftfeuchtigkeit** (`Tavg * Humidity`) einen zusätzlichen Einfluss auf die Feinstaubbelastung hat. Idee dahinter:

- Temperatur und Luftfeuchtigkeit beeinflussen gemeinsam viele chemische Reaktionen in der Atmosphäre
- Bei hoher Luftfeuchtigkeit kann z. B. Feinstaub stärker „gebunden“ oder verteilt werden – das hängt aber vom Temperaturverlauf ab


In [ ]:
# Interaktionsterm: Temperatur * Luftfeuchtigkeit
df_hamburg_pm25['Tavg_Humidity'] = df_hamburg_pm25['Tavg'] * df_hamburg_pm25['Humidity']

# Neues Feature-Set
X_inter = df_hamburg_pm25[['Tavg', 'Tavg_squared', 'Humidity', 'Log_Prcp', 'Wspd', 'Tavg_Humidity']]
y = df_hamburg_pm25['Pm25']

# Modell mit Interaktionsterm
X_inter = sm.add_constant(X_inter)
model_inter = sm.OLS(y, X_inter).fit()
print(model_inter.summary())

#### Ergebnis:

- Der Interaktionsterm `Tavg_Humidity` wurde dem Modell hinzugefügt.
- Der zugehörige **p-Wert liegt bei 0.76**, d. h. die Variable ist **nicht signifikant**.
- Auch die Modellgüte bleibt unverändert:
  - $R^2$ = 0.122 (wie im Modell ohne Interaktion)
  - AIC und BIC sind minimal **gestiegen**
- Fazit: Diese Interaktion liefert **keinen Mehrwert** für das Modell und kann wieder entfernt werden.

➡️ Dieser Schritt zeigt, wie einen Kombination von Features prinzipiell funktioniert, aber auch, dass nicht jede theoretisch plausible Kombination von Features ein Modell auch wirklich verbessert.


## 3.4. Zeitvariablen

Neben numerischen Transformationen und Interaktionstermen ist auch die **zeitliche Einordnung von Daten** ein wichtiger Bestandteil des Feature Engineerings. In diesem Schritt wird die ursprüngliche Monatsinformation (`Month`) genutzt, um daraus eine neue kategoriale Variable `Season` (Jahreszeit) zu erzeugen.

Hintergrund: Die Feinstaubbelastung unterliegt typischerweise **saisonalen Schwankungen**, etwa durch Heizemissionen im Winter, Inversionswetterlagen oder erhöhte Luftumwälzung im Sommer. Durch die Einbindung dieser Information als Modellfeature lassen sich solche Effekte gezielt abbilden.

Dazu wird `Season` in **Dummy-Variablen** umgewandelt, die in das Regressionsmodell einfließen. Auf diese Weise kann untersucht werden, ob sich die PM2.5-Konzentration **systematisch zwischen den Jahreszeiten unterscheidet** – und ob diese Zeitinformation zur Verbesserung der Modellgüte beiträgt.

Zur Einbindung der Zeitvariable `Season` in das Regressionsmodell wird die kategoriale Variable mithilfe von **One-Hot-Encoding** in **Dummy-Variablen** umgewandelt. Dabei gilt:

Jede Ausprägung einer kategorialen Variable wird zu einer eigenen Spalte, die nur den Wert `1` trägt, wenn die Zeile zu dieser Kategorie gehört – sonst `0`.

Beispiel für 4 Jahreszeiten:

| Season   | Winter | Frühling | Sommer | Herbst |
|----------|--------|----------|--------|--------|
| Sommer   | 0      | 0        | 1      | 0      |
| Herbst   | 0      | 0        | 0      | 1      |
| Winter   | 1      | 0        | 0      | 0      |
| Frühling | 0      | 1        | 0      | 0      |

Allerdings sollte in Regressionsmodellen **eine Kategorie ausgelassen** werden (hier: **Frühling**). Sie dient als **Referenzkategorie**. Dadurch vermeidet man **Multikollinearität**, also mathematische Redundanz im Modell. Das bedeutet:

- Die Effekte aller anderen Kategorien (Herbst, Sommer, Winter) werden im **Vergleich zum Frühling** berechnet.
- Beispiel: Ein Koeffizient von `-8.94` für Herbst bedeutet, dass die PM2.5-Konzentration im Herbst im Durchschnitt **8.94 µg/m³ niedriger ist als im Frühling**, wenn alle anderen Einflussfaktoren konstant gehalten werden.

In [ ]:
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Frühling'
    elif month in [6, 7, 8]:
        return 'Sommer'
    else:
        return 'Herbst'

df_hamburg_pm25.loc[:, 'Season'] = df_hamburg_pm25['Month'].apply(get_season)

df_hamburg_pm25.head()

In [ ]:
# Saison in Dummy-Variablen umwandeln (Frühling als Referenzkategorie)
season_dummies = pd.get_dummies(df_hamburg_pm25['Season'], drop_first=True, dtype=int)

In [ ]:
# Basis-Features
features_base = df_hamburg_pm25[['Tavg', 'Tavg_squared', 'Humidity', 'Log_Prcp', 'Wspd']]

# Kombinieren mit Saison-Dummies
X_season = pd.concat([features_base, season_dummies], axis=1)
y = df_hamburg_pm25['Pm25']

# Modell aufbauen und ausgeben
X_season = sm.add_constant(X_season)
model_season = sm.OLS(y, X_season).fit()
print(model_season.summary())


# 4. Fazit: Wirkung von Feature Engineering auf das Regressionsmodell

In diesem Notebook wurde schrittweise untersucht, wie sich gezielte Maßnahmen des **Feature Engineerings** auf die Vorhersagekraft eines linearen Regressionsmodells zur Feinstaubbelastung (PM2.5) auswirken. Ausgangspunkt war ein einfaches Modell mit fünf Wettervariablen, das lediglich **7,2 % der Varianz** erklären konnte.

Durch systematische Erweiterung und Optimierung der Features wurde das Modell deutlich verbessert:

| Maßnahme                          | Verbesserung                                   |
|----------------------------------|------------------------------------------------|
| Entfernen nicht signifikanter Variablen (`Pres`) | Kein Informationsverlust, schlankeres Modell |
| Log-Transformation (`log(Prcp + 1)`)            | Bessere Modellanpassung durch Stabilisierung der Skala |
| Quadratischer Term (`Tavg²`)     | Erfassung nichtlinearer Effekte, spürbare Steigerung von $R^2$ |
| Zeitvariable (`Season` als Dummy) | Deutlicher Leistungszuwachs, klare saisonale Muster |
| $R^2$ gesamt                 | von **0.072** → **0.176** (+145 %)             |

Zusätzlich wurde gezeigt, dass **nicht jede Feature-Kombination sinnvoll ist**: Ein getesteter Interaktionsterm (`Tavg * Humidity`) hatte keinen signifikanten Einfluss und wurde wieder entfernt.

➡️ Insgesamt zeigt dieses Notebook, wie **datengetriebenes Feature Engineering** schrittweise zu einem leistungsfähigeren und interpretierbareren Modell führen kann – eine Kernkompetenz in der datenanalytischen Praxis.


#### Ergebnis:

- Das Modell verbessert sich deutlich:
  - $R^2$: von **0.122** → **0.176**
  - Adj. $R^2$: von **0.120** → **0.174**
  - AIC und BIC sinken ebenfalls deutlich
- Alle Jahreszeiten sind **signifikant unterschiedlich** zum Frühling (Referenzkategorie):
  - **Sommer**: starke Reduktion der PM2.5-Werte (-16.68 µg/m³)
  - **Herbst**: ebenfalls reduzierte Werte (-8.94 µg/m³)
  - **Winter**: leicht erhöhte Werte (+3.17 µg/m³)

➡️ Diese Ergebnisse bestätigen, dass **saisonale Faktoren einen substantiellen Einfluss auf die Luftqualität** haben und im Modell berücksichtigt werden sollten.

## Notiz für Mareike
Für Hamburg doch nochmal LOWESS für alle Wettervariablen anschauen, vielleicht wurde noch was übersehen:

In [ ]:
# Liste der Variablen für die Analyse
variables = ['Tavg', 'Humidity', 'Prcp', 'Wspd', 'Pres']

# Erstelle Scatterplots mit LOWESS-Glättung
plt.figure(figsize=(12, 10))

for i, var in enumerate(variables, 1):
    plt.subplot(2, 3, i)  # 2 Zeilen, 3 Spalten
    sns.scatterplot(x=df_hamburg_pm25[var], y=df_hamburg_pm25['Pm25'], alpha=0.6)
    
    # LOWESS-Glättung anwenden
    lowess_result = lowess(df_hamburg_pm25['Pm25'], df_hamburg_pm25[var], frac=0.3)
    plt.plot(lowess_result[:, 0], lowess_result[:, 1], color='red', label='LOWESS')
    
    plt.title(f'LOWESS: PM2.5 vs {var}')
    plt.xlabel(var)
    plt.ylabel('PM2.5 (µg/m³)')
    plt.grid(True)
    plt.legend()

plt.tight_layout();

# 5. Bonus: Weitere Luftschadstoffe als erklärende Variablen

Im bisherigen Verlauf des Notebooks lag der Fokus auf der Frage, wie sich Wetterbedingungen und saisonale Effekte auf die Feinstaubbelastung (PM2.5) auswirken. Ein weiterer sinnvoller Analyseansatz besteht darin, zu prüfen, ob auch **andere Luftschadstoffe** (z. B. NO₂, SO₂, CO, O₃) dabei helfen können, PM2.5-Konzentrationen besser vorherzusagen.

Hintergrund: Viele Luftschadstoffe stehen miteinander in Zusammenhang – sie entstehen gemeinsam (z. B. durch Verkehr oder Industrie) oder beeinflussen sich chemisch gegenseitig. Durch die Einbindung dieser Variablen lässt sich testen, ob ein **Mehrkomponentenmodell** die Modellgüte weiter verbessern kann.

In diesem Abschnitt wird daher das bestehende Regressionsmodell um weitere Schadstoffvariablen erweitert ()`No2`, `O3`, `So2`). Diese Schadstoffe stehen in der Realität oft in engem Zusammenhang mit PM2.5, da sie aus ähnlichen Quellen stammen oder in gemeinsamen atmosphärischen Prozessen auftreten. Ziel ist es, den Erklärungswert des Modells zu steigern und gleichzeitig mögliche Wechselwirkungen zwischen den Schadstoffen sichtbar zu machen.


#### 🚫 Hinweis zur Datenqualität: CO-Konzentration

Bei der explorativen Untersuchung der Beziehung zwischen PM2.5 und Kohlenmonoxid (CO) fiel auf, dass in den vorliegenden Daten **ausschließlich der konstante Wert 0.10** für CO eingetragen ist. Dies ist physikalisch unrealistisch und spricht dafür, dass es sich um:

- eine Füllkonstante
- oder eine fehlerhafte Messreihe

handelt. Aufgrund der fehlenden Varianz ist CO **für die Regressionsanalyse nicht sinnvoll nutzbar**. Die ursprünglich im Modell enthaltene hohe Koeffizienz von CO (über 400) ist damit wahrscheinlich **ein Artefakt** der schlechten Datenqualität.

➡️ Die Variable CO wird im erweiterten Modell ausgeschlossen.

In [ ]:
# Relevante Spalten aus df_hamburg holen
pollutants = df_hamburg[['No2', 'O3', 'So2']]

# Schadstoffe in df_hamburg_pm25 hinzufügen
df_hamburg_pm25_extended = df_hamburg_pm25.copy()
df_hamburg_pm25_extended[['No2', 'O3', 'So2']] = pollutants

In [ ]:
df_hamburg_pm25_extended.head()

In [ ]:
df_hamburg_pm25_extended.isna().sum()

In [ ]:
season_dummies = pd.get_dummies(df_hamburg_pm25_extended['Season'], drop_first=True, dtype=int)


In [ ]:
# Kombiniertes DataFrame mit allen Features und Ziel
df_model = pd.concat([
    df_hamburg_pm25_extended[['Tavg', 'Tavg_squared', 'Humidity', 'Log_Prcp', 'Wspd', 'No2', 'O3', 'So2', 'Pm25']],
    season_dummies
], axis=1)

# Nur vollständige Zeilen behalten
df_model = df_model.dropna()


In [ ]:
X_final = df_model.drop(columns='Pm25')
y_final = df_model['Pm25']

# Konstanten-Term hinzufügen
X_final = sm.add_constant(X_final)

# Modell trainieren
model_final = sm.OLS(y_final, X_final).fit()
print(model_final.summary())



#### Ergebnis:

- Das erweiterte Modell zeigt eine deutliche Leistungssteigerung:
  - \( R^2 \): von **0.176** → **0.283**
  - Adj. \( R^2 \): von **0.174** → **0.280**
  - AIC und BIC sinken deutlich
- Alle hinzugefügten Schadstoffe sind **statistisch signifikant**, mit plausiblen Wirkungsrichtungen:
  - **NO₂, SO₂**: positiv korreliert mit PM2.5
  - **O₃**: negativer Zusammenhang
- Die Windgeschwindigkeit (`Wspd`) verliert im erweiterten Modell an Signifikanz – möglicherweise, weil ihre Wirkung durch die Schadstoffkonzentrationen überlagert wird

➡️ Die Einbindung weiterer Luftschadstoffe als erklärende Variablen erhöht die **Modellgüte substanziell** und bietet zusätzliche Einsichten in komplexe Zusammenhänge zwischen Umweltfaktoren und Feinstaubbelastung.


Da die Zusammenhänge zwischen Feinstaubbelastung und den anderen Schadstoffen nicht notwenigerweise linear sind, soll diese Annahme mit LOWESS-Regression überprüft werden.

In [ ]:
# Liste der Variablen für die Analyse
variables = ['So2', 'No2', 'O3']

# Erstelle Scatterplots mit LOWESS-Glättung
plt.figure(figsize=(12, 10))

for i, var in enumerate(variables, 1):
    plt.subplot(2, 3, i)  # 2 Zeilen, 3 Spalten
    sns.scatterplot(x=df_hamburg_pm25_extended[var], y=df_hamburg_pm25_extended['Pm25'], alpha=0.6)
    
    # LOWESS-Glättung anwenden
    lowess_result = lowess(df_hamburg_pm25_extended['Pm25'], df_hamburg_pm25_extended[var], frac=0.3)
    plt.plot(lowess_result[:, 0], lowess_result[:, 1], color='red', label='LOWESS')
    
    plt.title(f'LOWESS: PM2.5 vs {var}')
    plt.xlabel(var)
    plt.ylabel('PM2.5 (µg/m³)')
    plt.grid(True)
    plt.legend()

# plt.ylim(0, df_hamburg_pm25_extended['Pm25'].max() * 1.1)

plt.tight_layout();

#### 🔁 Feature-Engineering für Ozon: Quadratischer Zusammenhang

Die visuelle Analyse der Beziehung zwischen PM2.5 und Ozon (`O3`) mithilfe von LOWESS zeigt eine deutliche **U-förmige Struktur**:  
- Bei **niedrigen und hohen Ozonwerten** ist die Feinstaubbelastung erhöht  
- Bei **mittleren Werten** hingegen am geringsten  
- Die Form wader Kurve ist rundlich, nicht spitz – also kein V, sondern ein **sanfter quadratischer Verlauf**

Um diesen nichtlinearen Zusammenhang im Regressionsmodell abzubilden, wird ein **quadratischer Term** `O3_squared` eingeführt.

In [ ]:
# Quadratischen Term für Ozon erstellen
df_model['O3_squared'] = df_model['O3'] ** 2


In [ ]:
X_quad_o3 = df_model.drop(columns=['Pm25']) # Features
y_quad_o3 = df_model['Pm25']

# Konstanten-Term hinzufügen
X_quad_o3 = sm.add_constant(X_quad_o3)

# Modell trainieren
model_quad_o3 = sm.OLS(y_quad_o3, X_quad_o3).fit()
print(model_quad_o3.summary())



#### ✅ Ergebnis: Quadratische Transformation von Ozon verbessert das Modell

Die zuvor mit LOWESS beobachtete **U-förmige Beziehung zwischen PM2.5 und O₃** konnte durch die Einführung eines quadratischen Terms (`O3_squared`) erfolgreich im Regressionsmodell abgebildet werden.

Das erweiterte Modell zeigt eine weitere Leistungssteigerung:

- $R^2$: von **0.283** → **0.313**
- Adj. $R^2$: von **0.280** → **0.310**

Beide Ozon-Terme (`O3`, `O3_squared`) sind **hoch signifikant** (p < 0.001). Die Koeffizienten deuten auf einen **nichtlinearen Zusammenhang** hin, bei dem die Feinstaubbelastung bei mittleren O₃-Werten am geringsten ist – und sowohl bei sehr niedrigen als auch sehr hohen Ozonwerten wieder ansteigt.

➡️ Diese Transformation verdeutlicht, wie wichtig es ist, **visuelle Datenanalyse** mit **gezieltem Feature Engineering** zu kombinieren, um komplexe Umweltzusammenhänge zu modellieren.


# 6. Finales Modell: Visualisierung der Ergebnisse

In [ ]:
# Vorhersagen erzeugen
y_pred = model_quad_o3.predict(X_quad_o3)

# Scatterplot
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_quad_o3, y=y_pred, alpha=0.5)
plt.plot([y_quad_o3.min(), y_quad_o3.max()],
         [y_quad_o3.min(), y_quad_o3.max()],
         color='red', linestyle='--', label='Ideal')
plt.xlabel('Tatsächliche PM2.5-Werte (µg/m³)')
plt.ylabel('Vorhergesagte PM2.5-Werte (µg/m³)')
plt.title('Vorhersage vs. tatsächliche Werte')
plt.legend()
plt.grid(True)
plt.tight_layout();


#### 📈 Vorhersage vs. tatsächliche Werte

Der Scatterplot zeigt die vom Modell vorhergesagten PM2.5-Werte im Vergleich zu den tatsächlich gemessenen Werten. Die rote Linie markiert den idealen Fall, in dem Vorhersage und Realität exakt übereinstimmen würden.

Beobachtungen:

- Im zentralen Bereich (ca. 20–80 µg/m³), wo die meisten Datenpunkte liegen, liegt ein Großteil der Punkte **unterhalb der Diagonalen** → das Modell **überschätzt die PM2.5-Werte** leicht.
- Ab etwa 80 µg/m³ Vorhersagewert steigt die Modellvorhersage weiter an – **obwohl die tatsächlichen Werte eine Obergrenze bei ca. 80 erreichen**.
- Daraus ergibt sich eine **systematische Überschätzung im oberen Wertebereich**, was darauf hindeutet, dass das Modell **keine natürliche Sättigung erkennt**.

➡️ Insgesamt liefert das Modell brauchbare Vorhersagen im Hauptbereich der Daten, tendiert jedoch dazu, **sehr hohe PM2.5-Werte zu überschätzen**, was bei linearen Modellen ohne Schranken ein bekanntes Phänomen ist.


In [ ]:
# Residuen berechnen
residuals = y_quad_o3 - y_pred

# Plot
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_pred, y=residuals, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Vorhergesagte PM2.5-Werte (µg/m³)')
plt.ylabel('Residuen')
plt.title('Residuen vs. Vorhersagen')
plt.grid(True)
plt.tight_layout();


#### 📉 Residuen vs. Vorhersagen

Der Residuenplot zeigt die Differenz zwischen tatsächlichem und vorhergesagtem PM2.5-Wert in Abhängigkeit vom Vorhersagewert. Idealerweise sollten sich die Punkte zufällig um die Null-Linie gruppieren – ohne erkennbares Muster.

- Die Punkte streuen relativ gleichmäßig um die Null-Linie → **kein systematischer Fehler** sichtbar.
- Kein klarer Trend in den Residuen → das Modell verzerrt die Vorhersagen nicht systematisch bei hohen oder niedrigen PM2.5-Werten.
- Die Varianz der Residuen scheint über den Wertebereich hinweg relativ konstant, das Kriterium der **Homoskedastizität** ist also weitgehend erfüllt.

➡️ Der Residuenplot unterstützt die Annahme, dass das Modell nicht nur gut angepasst ist, sondern auch robust und stabil im Fehlerverhalten.

#### 🔎 Auffälligkeit im Residuenplot: diagonale Streifen

Beim Residuenplot zeigen sich insbesondere im negativen Bereich der Residuen **schräg verlaufende Streifenmuster**. Diese diagonalen Anordnungen entstehen häufig, wenn:

- Eingabevariablen (z. B. Ozon oder NO₂) nur wenige diskrete Werte annehmen
- das Modell daraus kontinuierliche Vorhersagen erzeugt
- die tatsächlichen PM2.5-Werte wiederum gerundet vorliegen

➡️ Das Streifenmuster ist in diesem Fall kein Hinweis auf Modellfehler, sondern eher auf die **Granularität und Struktur der Eingangsdaten** zurückzuführen.




# 7. Modellvergleich

| Modell | Enthaltene Features | $R^2$ | Adj. $R^2$ | AIC | BIC | Bemerkung |
|--------|----------------------|----------|----------------|-----|-----|-----------|
| **1** | Wetter (linear) | 0.072 | 0.070 | ~25280 | ~25310 | Ausgangsmodell |
| **2** | Wetter (ohne Luftdruck) | 0.072 | 0.070 | ~25280 | ~25310 | `Pres` entfernt – kein Informationsverlust |
| **3** | + `log(Prcp + 1)` | 0.078 | 0.076 | ~25260 | ~25290 | Nichtlineare Transformation |
| **4** | + `Tavg²` | 0.122 | 0.120 | ~25120 | ~25160 | Quadratische Temperatur |
| **5** | + `Season` (Dummy) | 0.176 | 0.174 | ~24950 | ~25000 | Zeitstruktur eingebunden |
| **6** | + Schadstoffe (ohne `Pm10`, mit `Co`) | 0.283 | 0.280 | ~22870 | ~22940 | Stark verbesserte Modellgüte |
| **7** | Schadstoffe ohne `Co`, + `O3²` | **0.313** | **0.310** | ~24020* | ~24100* | Bestes Modell, plausibler Zusammenhang |

> \* Die AIC/BIC-Werte in Modell 7 sind durch mehr Beobachtungen nicht direkt vergleichbar mit Modell 6 (siehe Hinweis im Text).

➡️ Der Modellvergleich zeigt, wie systematisches Feature Engineering die Vorhersagegüte deutlich steigern kann:  

Durch die Kombination aus Transformationen, nichtlinearen Erweiterungen und der Einbindung zeitlicher sowie inhaltlich verwandter Einflussfaktoren konnte das Regressionsmodell von einem ursprünglichen $R^2§ von **0.072** auf **0.313** verbessert werden – eine Steigerung um über **330 %**.

